# Notes
- Use pip freeze to generate requirements.txt
- Group data for each year (SQL select) into a dataframe

# Requirements

* Query the dataset using sqlite. Only load the final dataset into a dataframe.

* Give an overview of the respondents of the survey. What is the sample size?
* What are the sociodemographic features of the respondents? Do you see any evidence of sampling bias?
* Perform exploratory data analysis. This should include creating statistical summaries and charts, checking for correlations and other relationships between variables, as well as other EDA elements.
* In a plot, report the prevalence rate of at least three mental diseases. (https://en.wikipedia.org/wiki/Prevalence)
* Make sure to plot the confidence interval and provide its interpretation.
* Your notebook should be readable as a standalone document. In Markdown cells inform the reader of the questions you are trying to answer, and provide an interpretation of your results.
* Provide suggestions about how your analysis can be improved.
# Questions to answer (general)

- What are the main types and subtypes of data?
- What are the main metrics of location? What are their main characteristics?
- What is variability? What are the main metrics of variability and their characteristics?
- What is a confidence interval? Why do we need it? Why is it not sufficient to just report the point estimates?
- What is correlation? How do we use it to analyze data?
- What is a contingency table?

# Plan of action

- Import data into a single dataframe, that is coherent (it makes sense looking at it)
- Review the data
- Clean the data
- Perform exploratory data analysis, main goal

Let's filter the data to only include the questions that are present in all years, as we are interested in the trends over time.
Also, let's clean the data by renaming the columns to lowercase and removing spaces, and renaming SurveyId to year as it is more intuitive.

In [38]:
import sqlite3
import pandas as pd
import helpers

conn = sqlite3.connect('mental_health.sqlite')

query = """
SELECT 
    s.SurveyID as year,  -- Renamed in the query itself
    s.Description as survey_description,
    a.UserID as user_id,
    a.QuestionID as question_id,
    q.QuestionText as question_text,
    a.AnswerText as answer_text
FROM Answer a
JOIN Question q ON a.QuestionID = q.QuestionID
JOIN Survey s ON a.SurveyID = s.SurveyID
"""

# Create initial dataframe
df = pd.read_sql_query(query, conn)

# Close connection
conn.close()

# Convert all column names to lowercase
df.columns = df.columns.str.lower()

df.head()

,year,survey_description,user_id,question_id,question_text,answer_text
0,2014,mental health survey for 2014,1,1,What is your age?,37
1,2014,mental health survey for 2014,2,1,What is your age?,44
2,2014,mental health survey for 2014,3,1,What is your age?,32
3,2014,mental health survey for 2014,4,1,What is your age?,31
4,2014,mental health survey for 2014,5,1,What is your age?,31


In [39]:
# Number of unique respondents per year
yearly_respondents = df.groupby('year')['user_id'].nunique()

# Show results directly (no print needed in PyCharm)
yearly_respondents

year
2014    1260
2016    1433
2017     756
2018     417
2019     352
Name: user_id, dtype: int64

In [40]:
# Clean age data by removing impossible values
age_df = df[df['question_text'] == 'What is your age?']
clean_age = pd.to_numeric(age_df['answer_text'], errors='coerce')
clean_age = clean_age[(clean_age >= 16) & (clean_age <= 80)]  # Reasonable age range
clean_age_stats = clean_age.describe()

clean_age_stats

count    4203.000000
mean       33.855817
std         8.068257
min        17.000000
25%        28.000000
50%        33.000000
75%        38.000000
max        74.000000
Name: answer_text, dtype: float64

In [41]:

# Let's also look at age distribution by year
age_by_year = age_df.copy()
age_by_year['clean_age'] = pd.to_numeric(age_by_year['answer_text'], errors='coerce')
age_by_year = age_by_year[(age_by_year['clean_age'] >= 18) & (age_by_year['clean_age'] <= 100)]
yearly_age_stats = age_by_year.groupby('year')['clean_age'].describe()

yearly_age_stats

,count,mean,std,min,25%,50%,75%,max
year,,,,,,,,
2014,1252.0,32.083866,7.289722,18.0,27.0,31.0,36.0,72.0
2016,1429.0,34.131561,8.263825,19.0,28.0,33.0,39.0,99.0
2017,754.0,34.988064,8.338051,18.0,29.0,34.0,40.0,67.0
2018,417.0,34.916067,8.047047,19.0,29.0,34.0,39.0,67.0
2019,351.0,35.595442,8.891819,19.0,29.0,34.0,41.0,64.0


Looking at the cleaned age statistics, let me help interpret:

### Sample Size and Distribution:
- Count: 4203 valid responses
- Mean: 33.86 years
- Median (50%): 33 years
- The mean and median being close suggests a relatively symmetric distribution


### Age Spread:

- Standard Deviation: 8.07 years
- IQR: 38 years (75th) - 28 years (25th) = 10 years
- Range: 17 years (min) to 74 years (max)

### Evidence of Sampling Bias:

- Age concentration: 50% of respondents are between 28-38 years
- Underrepresentation of:
    - Senior tech workers (40+ years)
    - Early career professionals (< 25 years)
    - The narrow standard deviation (8.07 years) suggests limited age diversity

Let's analyze other demographics to get a fuller picture.

In [42]:
# Gender distribution
gender_dist = helpers.get_responses_by_question(df, 'What is your gender?')
gender_dist

answer_text,-1,43,A little about you,AFAB,Agender,Agender trans woman,Agender/genderfluid,All,Androgyne,Androgynous,...,"ostensibly male, unsure what that really means",p,queer,queer/she/they,rr,something kinda male?,sometimes,trans woman,transgender,uhhhhhhhhh fem genderqueer?
year,,,,,,,,,,,,,,,,,,,,,
2014,0,0,1,0,1,0,0,1,1,0,...,1,1,1,1,0,1,0,0,0,0
2016,3,0,0,1,2,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2017,13,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,1,0,1
2018,3,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2019,5,1,0,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [44]:

# Location/Country distribution
location_dist = helpers.get_responses_by_question(df, 'What country do you live in?')
location_dist.sum().nlargest(5)
# Top 5 countries

answer_text
United States of America    1853
United States                751
United Kingdom               482
Canada                       199
Germany                      136
dtype: int64

In [45]:
# Company size distribution
company_size_dist = helpers.get_responses_by_question(df, 'How many employees does your company or organization have?')
company_size_dist

answer_text,-1,1-5,100-500,26-100,500-1000,6-25,More than 1000
year,,,,,,,
2014,0,162,176,289,61,290,282
2016,287,60,248,292,80,210,256
2017,113,20,203,128,48,86,158
2018,56,5,81,70,31,69,105
2019,48,7,80,45,27,34,111
